In [ ]:
! pip install datasets transformers rouge-score nltk SentencePiece
! pip install --upgrade pyyaml

     |████████████████████████████████| 264 kB 5.2 MB/s 
     |████████████████████████████████| 2.6 MB 10.2 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 118 kB 70.2 MB/s 
     |████████████████████████████████| 243 kB 53.7 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 636 kB 70.6 MB/s 
     |████████████████████████████████| 895 kB 62.7 MB/s 
     |████████████████████████████████| 3.3 MB 73.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install hf-lfs
!git config --global user.email "jakerutherford@outlook.com"
!git config --global user.name "JakeMSc"

     |████████████████████████████████| 4.0 MB 5.2 MB/s 


In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: JakeMSc
Password: 
Login successful
Your token: hAZfREWUQijkQBHfmbnHEIPIwxTDREXbyBkcGueDblJagPmuXLRagIJaRUiHNaEcQhywCGvKExdpEQRVATeamWsVUOPitqJWcWZygoRNmGHzenCuNuzXqHCPwBjBIAid 

Your token has been saved to /root/.huggingface/token


In [ ]:
model_checkpoint = "t5-small"

model_name = model_checkpoint.split("/")[-1] + "-finetuned-ts-and-cs-v2"
!transformers-cli repo create {model_name}

2021-08-15 18:47:26.384582: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/usr/local/lib/python3.7/dist-packages/transformers/commands/user.py:269: UserWarning: Managing repositories through transformers-cli is deprecated. Please use `huggingface-cli` instead.
  "Managing repositories through transformers-cli is deprecated. Please use `huggingface-cli` instead."
git version 2.17.1
git-lfs/2.13.3 (GitHub; linux amd64; go 1.16.2; git a5e65851)

You are about to create JakeMSc/t5-base-finetuned-ts-and-cs-v2
Proceed? [Y/n] y

Your repo now lives at:
  https://huggingface.co/JakeMSc/t5-base-finetuned-ts-and-cs-v2

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/JakeMSc/t5-base-finetuned-ts-and-cs-v2



## Load Data
Loading TOSDR.org data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
from datasets import load_dataset

dataset_file = 'drive/MyDrive/Colab_Notebooks/train_tosdr_prepared.csv'

dataset = load_dataset('csv', data_files=dataset_file, split='train')

dataset = dataset.train_test_split(test_size=0.1)

train_dataset = dataset['train']
val_dataset = dataset['test']

Using custom data configuration default-a4a77547fb39f821
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-a4a77547fb39f821/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

##Tokenize

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained("t5-small")

loading configuration file https://huggingface.co/google/pegasus-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3fa0446657dd3714a950ba400a3fa72686d0f815da436514e4823a973ef23e20.7a0cb161a6d34c3881891b70d4fa06557175ac7b704a19bf0100fb9c21af9286
Model config PegasusConfig {
  "_name_or_path": "google/pegasus-large",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token

In [ ]:
max_input_length = 512
max_target_length = 128

if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets['train']['document'][0]

'cookies are small data files that are commonly stored on your device when you browse and use websites and online services. they are widely used to make websites work or to work more efficiently as well as to provide reporting information and assist with service or advertising personalization. cookies are not the only types of technology that enable this functionality. we also use other similar types of technologies. see below for more information and examples. what are advertising identifiers. advertising identifiers are similar to cookies and are found on many mobile devices and tablets for example the identifier for advertisers or idfa on apple ios devices and the google advertising id on android devices and certain streaming media devices. like cookies advertising identifiers are used to make online advertising more relevant. why does netflix use cookies and advertising identifiers. essential cookies these cookies are strictly necessary to provide our website or online service. for

## Fine-tuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch 

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, use_auth_token=True).to(torch_device)

loading configuration file https://huggingface.co/google/pegasus-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3fa0446657dd3714a950ba400a3fa72686d0f815da436514e4823a973ef23e20.7a0cb161a6d34c3881891b70d4fa06557175ac7b704a19bf0100fb9c21af9286
Model config PegasusConfig {
  "_name_or_path": "google/pegasus-large",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token

In [ ]:
batch_size = 1
args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_accumulation_steps=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    adafactor=True,
    push_to_hub=True,
    push_to_hub_model_id=model_name,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

nltk.download('punkt')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: Unnamed: 0, document, url, summary.
***** Running training *****
  Num examples = 291
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 2910


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.261700,2.136362,26.216300,8.457000,21.851500,21.777000,32.575800
2,2.443900,1.876113,27.956200,10.940300,24.911700,24.870600,31.636400
3,2.043000,1.727877,32.238700,14.504700,27.846500,27.941400,27.363600
4,1.348000,1.678856,32.551600,15.971000,27.586500,27.864700,27.545500
5,1.154200,1.623587,33.370800,16.769400,28.323200,28.220300,22.060600
6,1.401000,1.585874,33.182900,16.953000,28.439800,28.101200,19.969700
7,1.301700,1.604203,40.062600,23.505600,35.892100,35.571700,17.909100
8,1.111600,1.575186,40.429000,24.604800,36.533500,36.172300,19.454500
9,1.816600,1.588958,38.484600,22.638400,34.185500,34.417300,19.484800
10,1.263000,1.593920,38.669000,22.776200,34.487400,34.670900,18.697000


The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: Unnamed: 0, document, url, summary.
***** Running Evaluation *****
  Num examples = 33
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: Unnamed: 0, document, url, summary.
***** Running Evaluation *****
  Num examples = 33
  Batch size = 1
The following columns in the evaluation set  don't have a corresponding argument in `PegasusForConditionalGeneration.forward` and have been ignored: Unnamed: 0,

TrainOutput(global_step=2910, training_loss=1.5935436279912993, metrics={'train_runtime': 1220.2479, 'train_samples_per_second': 2.385, 'train_steps_per_second': 2.385, 'total_flos': 772113425448960.0, 'train_loss': 1.5935436279912993, 'epoch': 10.0})

## Save model

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to ./results
Configuration saved in ./results/config.json
Model weights saved in ./results/pytorch_model.bin
tokenizer config file saved in ./results/tokenizer_config.json
Special tokens file saved in ./results/special_tokens_map.json


'https://huggingface.co/JakeMSc/pegasus-large-finetuned-ts-and-cs-v2/commit/ad17d496d316adb9cd305dda4e9fb7e25a17fa34'

## Testing model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_checkpoint = "google/pegasus-large"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

my_model = 'JakeMSc/pegasus-large-legal-dapt-finetuned-ts-and-cs'

#tokenizer = AutoTokenizer.from_pretrained(my_model)  
model = AutoModelForSeq2SeqLM.from_pretrained(my_model, use_auth_token=True).to(torch_device)

loading configuration file https://huggingface.co/google/pegasus-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3fa0446657dd3714a950ba400a3fa72686d0f815da436514e4823a973ef23e20.7a0cb161a6d34c3881891b70d4fa06557175ac7b704a19bf0100fb9c21af9286
Model config PegasusConfig {
  "_name_or_path": "google/pegasus-large",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 1,
  "force_bos_token

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

storing https://huggingface.co/JakeMSc/pegasus-large-legal-dapt-finetuned-ts-and-cs/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/543b81b673aa12425582b9bbc273c4951adbc85ef6dd9b20fc24956e70e17a86.d8c50a847bc0bb42d1045118a440bb3fccaba0511d796ce881a0e1d53b59e98a
creating metadata file for /root/.cache/huggingface/transformers/543b81b673aa12425582b9bbc273c4951adbc85ef6dd9b20fc24956e70e17a86.d8c50a847bc0bb42d1045118a440bb3fccaba0511d796ce881a0e1d53b59e98a
loading weights file https://huggingface.co/JakeMSc/pegasus-large-legal-dapt-finetuned-ts-and-cs/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/543b81b673aa12425582b9bbc273c4951adbc85ef6dd9b20fc24956e70e17a86.d8c50a847bc0bb42d1045118a440bb3fccaba0511d796ce881a0e1d53b59e98a
All model checkpoint weights were used when initializing PegasusForConditionalGeneration.

All the weights of PegasusForConditionalGeneration were initialized from the model checkpoint at JakeMSc/pega

In [ ]:
print("input:")
input_text = input()

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(torch_device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(torch_device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids=source_ids,
        attention_mask=source_mask,
        max_length=250,
        num_beams=9,
        length_penalty=5.0, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)